In [1]:
import csv
import itertools as it
import numpy as np
np.random.seed(0)
import lab_util
import sys


In [2]:

data = []
n_positive = 0
n_disp = 0
with open("./reviews.csv") as reader:
    csvreader = csv.reader(reader)
    next(csvreader)
    for id, review, label in csvreader:
        label = int(label)
        # hacky class balancing
        if label == 1:
            if n_positive == 2000:
                continue
            n_positive += 1
        if len(data) == 4000:
            break
        data.append((review, label))

        if n_disp > 5:
            continue
        n_disp += 1
        # print("review:", review)
        # print("rating:", label, "(good)" if label == 1 else "(bad)")
        # print()

# print(f"Read {len(data)} total reviews.")
np.random.shuffle(data)
reviews, labels = zip(*data)
train_reviews = reviews[:3000]
train_labels = labels[:3000]
val_reviews = reviews[3000:3500]
val_labels = labels[3000:3500]
test_reviews = reviews[3500:]
test_labels = labels[3500:]

In [3]:
# hmm model
class HMM(object):
    def __init__(self, num_states, num_words):
        self.num_states = num_states
        self.num_words = num_words

        self.states = range(num_states)
        self.symbols = range(num_words)

        # initialize the matrix A with random transition probabilities p(j|i)
        # A should be a matrix of size `num_states x num_states`
        # with rows that sum to 1
        self.A = np.random.random([self.num_states, self.num_states])
        self.A = self.A / np.sum(self.A, axis=1).reshape(-1, 1) # your code here

        # initialize the matrix B with random emission probabilities p(o|i)
        # B should be a matrix of size `num_states x num_words`
        # with rows that sum to 1
        self.B = np.random.random([self.num_states, self.num_words])
        self.B = self.B / np.sum(self.B, axis=1).reshape(-1, 1)
        # your code here

        # initialize the vector pi with a random starting distribution
        # pi should be a vector of size `num_states`
        self.pi = np.random.random(self.num_states)
        self.pi = self.pi / np.sum(self.pi) 
        # your code here

    def generate(self, n):
        """randomly sample the HMM to generate a sequence.
        """
        # we'll give you this one

        sequence = []
        # initialize the first state
        state = np.random.choice(self.states, p=self.pi)
        for i in range(n):
            # get the emission probs for this state
            b = self.B[state, :]
            # emit a word
            word = np.random.choice(self.symbols, p=b)
            sequence.append(word)
            # get the transition probs for this state
            a = self.A[state, :]
            # update the state
            state = np.random.choice(self.states, p=a)
        return sequence

    def forward(self, obs):
        # run the forward algorithm
        # this function should return a `len(obs) x num_states` matrix
        # where the (i, j)th entry contains p(obs[:t], hidden_state_t = i)

        alpha = np.zeros((len(obs), self.num_states))
        # your code here!
        # print(self.pi)
        alpha[0, :] = self.pi * self.B[:, obs[0]]
        # print(alphal[0,:])
        for t in range(1, len(obs)):
            for j in range(self.num_states):
            # alpha[t, j] = np.dot(alpha[t-1, :], self.A[:, j]) * self.B[j, obs[t]]
                alpha[t, j] = alpha[t-1].dot(self.A[:, j]) * self.B[j, obs[t]]
        alpha[alpha == 0] = sys.float_info.min
        alpha = np.log(alpha)

        return alpha

    def backward(self, obs):
        # run the backward algorithm
        # this function should return a `len(obs) x num_states` matrix
        # where the (i, j)th entry contains p(obs[t+1:] | hidden_state_t = i)

        beta = np.zeros((len(obs), self.num_states))
        # your code here!
        beta[len(obs) - 1, :] = np.ones(self.num_states)
        for t in range(len(obs)-2, -1, -1):
            for j in range(self.num_states):
                beta[t, j] = (beta[t+1, :] * self.B[:, obs[t+1]]).dot(self.A[j, :])

        beta[beta == 0] = sys.float_info.min
        beta = np.log(beta)

        return beta
        
    def forward_backward(self, obs):
        # compute forward--backward scores

        # logprob is the total log-probability of the sequence obs 
        # (marginalizing over hidden states)

        # gamma is a matrix of size `len(obs) x num_states`
        # it contains the marginal probability of being in state i at time t

        # xi is a tensor of size `len(obs) x num_states x num_states`
        # it conains the marginal probability of transitioning from i to j at t

        # your code here!
        log_alpha = self.forward(obs)
        log_beta = self.backward(obs)

        # logprob = np.logaddexp(log_alpha[-1, 0], log_alpha[-1, 1])
        # if self.num_states > 2:
        #   for i in range(2, self.num_states):
        #     logprob = np.logaddexp(log)
        # print(log_alpha[-1, :])

        logprob = np.log(np.sum(np.exp(log_alpha[-1, :])))
        # print(logprob)
        # prob1 = 0
        # for i in range(self.num_states):
        #   prob1 += self.pi[i] * self.B[i, obs[0]] * np.exp(log_beta[0, i])
        # print(np.log(prob1))
        log_xi = np.zeros((len(obs)-1, self.num_states, self.num_states))
        gamma = np.zeros((len(obs), self.num_states))

        for t in range(len(obs)-1):
            for i in range(self.num_states):
                for j in range(self.num_states):
                    log_xi[t, i, j] = log_alpha[t, i] + np.log(self.A[i, j]) + np.log(self.B[j, obs[t+1]]) + log_beta[t+1, j] - logprob
        xi = np.exp(log_xi)
        # print(xi)
        gamma = np.sum(xi, axis=2)
        gamma = np.vstack((gamma, np.sum(xi[len(obs) - 2, :, :], axis=1).reshape(1, -1)))

        pi_new = gamma[0, :] 
        pi_new[pi_new == 0]= sys.float_info.min
        self.pi = pi_new / np.sum(pi_new)

        return logprob, xi, gamma

    def learn_unsupervised(self, corpus, num_iters):
        """Run the Baum Welch EM algorithm
        """

        for i_iter in range(num_iters):
            # your code here
            expected_l_si = np.zeros(self.num_states) 
            expected_l_sij = np.zeros((self.num_states, self.num_states)) 
            total_logprob = 0
            B_new_numerator = np.zeros((self.num_states, self.num_words))
            B_new_denomenator = np.zeros((self.num_states, self.num_words))
            for review in corpus:
                logprob, xi, gamma = self.forward_backward(review)
                # your code here 
                
                expected_sij = np.sum(xi, axis=0)
                # print(expected_sij)
                expected_si = np.sum(gamma[:-1, :], axis=0)
                expected_l_si = expected_l_si + expected_si
                expected_l_sij = expected_l_sij + expected_sij
                total_logprob += logprob
                # print(expected_si)
                # print(np.sum(expected_sij, axis=1))

                for j in range(self.num_states): #states
                    for symbol in range(self.num_words): #seq
                        indices = [idx for idx, val in enumerate(review) if val == symbol]
                        numerator_b = sum(gamma[indices, j])
                        denomenator_b = sum(gamma[:, j] )

                        B_new_numerator[j, symbol] = numerator_b
                        B_new_denomenator[j, symbol] = denomenator_b
                # print(logprob)
            print("log-likelihood", total_logprob)
            # your code here
            # print(expected_l_sij)
            # print(expected_l_si)
            A_new = expected_l_sij / expected_l_si.reshape(-1, 1)
            A_new[A_new == 0] = sys.float_info.min

            B_new = np.zeros((self.num_states, self.num_words))
            for j in range(self.num_states): #states
                  for symbol in range(self.num_words):
                        if B_new_denomenator[j, symbol] == 0:
                            B_new[j, symbol] = 0
                        else:
                            B_new[j, symbol] = B_new_numerator[j, symbol] / B_new_denomenator[j, symbol]
            B_new[B_new == 0] = sys.float_info.min
            # print(B_new.shape)
            # print(np.sum(A_new, axis=1))
            self.A = A_new
            self.B = B_new

In [4]:
tokenizer = lab_util.Tokenizer()
tokenizer.fit(train_reviews)
train_reviews_tk = tokenizer.tokenize(train_reviews)
print(tokenizer.vocab_size)
# print(len(train_reviews_tk))
hmm = HMM(num_states=50, num_words=tokenizer.vocab_size)
hmm.learn_unsupervised(train_reviews_tk, num_iters=5)

# import pickle
# with open("/content/6864-hw1/hmm_r2_s10_n10.txt", "wb") as fp:
#   pickle.dump(hmm, fp, -1)

2006
log-likelihood -1457884.7228594148
log-likelihood -2112935.295431081
log-likelihood -2112934.401801285
log-likelihood -2112933.551055862
log-likelihood -2112932.9250883325


In [10]:
for i in range(hmm.num_states):
    print(np.sum(hmm.B[i, :]))
    most_probable = np.argsort(hmm.B[i, :])[-10:]
    print(f"state {i}")
    for o in most_probable:
        print(tokenizer.token_to_word[o], hmm.B[i, o])
    print()

0.9999999999999999
state 0
than 0.03051550900008715
amounts 0.0307816689494252
i 0.03486354478620912
to 0.03930483951442974
of 0.04404783177827235
that 0.07144674007185217
a 0.07846087894929311
. 0.09765691750799399
<unk> 0.1167721775811637
, 0.15053890651878618

0.9999999999999998
state 1
a 0.03575000842324872
tasted 0.03753353415943841
drink 0.040050828760549245
to 0.043103344784770375
did 0.04377596283012293
sandwiches 0.04380082757065057
. 0.0752546233466296
<unk> 0.0830077631985694
it 0.0836253369521685
, 0.1120732943854877

1.0
state 2
drink 0.03783391162380982
using 0.03802657757529539
salad 0.04794739328889711
to 0.04933921822882991
vinegar 0.055499026776360524
. 0.05840110460304444
i 0.06187833756956893
of 0.062061666971077414
a 0.12144497272387937
, 0.12308891869932204

1.0000000000000004
state 3
this 0.033407287241896545
i 0.03594345027074527
dressing 0.053252519359956396
so 0.053923789948647924
. 0.055836497286819414
but 0.06441534917399176
it 0.06861663735427871
than 0.080

In [7]:
for i in range(10):
    print(tokenizer.de_tokenize([hmm.generate(10)]))

['so closer a really this using sandwiches so did ,']
['love . <unk> got to a it on this no']
['sandwiches , to <unk> but bit , to it it']
['i a was on using amounts amounts it salad i']
['i . , vinegar of of but but to i']
['i love <unk> <unk> . amounts . like a on']
['vinegar hard but did was a , sandwiches <unk> i']
['tasted , but it . using like so really vinegar']
['i i than go but hard a closer love but']
['go . tasted this sandwiches got did on to to']


In [13]:
def train_model(xs_featurized, ys):
    import sklearn.linear_model
    model = sklearn.linear_model.LogisticRegression()
    model.fit(xs_featurized, ys)
    return model

def eval_model(model, xs_featurized, ys):
    pred_ys = model.predict(xs_featurized)
    print("test accuracy", np.mean(pred_ys == ys))

def training_experiment(name, featurizer, n_train):
    print(f"{name} features, {n_train} examples")
#     hmm_featurizer(tokenizer.tokenize([train_reviews[:n_train][0]]))
#     print(tokenizer.tokenize([train_reviews[:n_train][0]]))
    train_xs = np.array([
        hmm_featurizer(tokenizer.tokenize([review])) 
        for review in train_reviews[:n_train]
    ])
    train_ys = train_labels[:n_train]
    test_xs = np.array([
        hmm_featurizer(tokenizer.tokenize([review]))
        for review in test_reviews
    ])
    test_ys = test_labels
    model = train_model(train_xs, train_ys)
    eval_model(model, test_xs, test_ys)
    print()

def hmm_featurizer(review):
    _, _, gamma = hmm.forward_backward(review[0])
    return gamma.sum(axis=0)

training_experiment("hmm", hmm_featurizer, n_train=3000)

hmm features, 3000 examples
test accuracy 0.544



In [14]:
import pickle
with open("./hmm_results/hmm_r_s50_n5.txt", "wb") as fp:
    pickle.dump(hmm, fp, -1)